## 课程目标

- 如何利用神经网络构建语言模型
- 神经网络构建语言模型的优势
- 对神经网络学到的模型进行可视化

## 神经网络与梯度下降

三个偏导数加起来就构成了 三个参数 的梯度  

从输出层往回传导数的过程：反向传播算法

纠错：视频1 13:34 $w_{21}x_2$ 应为 $w_{21}x_1$

### 作业1

基于矩阵乘法，用 Tensorflow 实现 Lecture 3 最后作业的单隐层神经网络

### 提示

In [3]:
import tensorflow as tf

In [4]:
W = tf.constant([[1, 1], [2, 2]])
x = tf.constant([[1], [2]])

In [5]:
W

<tf.Tensor 'Const:0' shape=(2, 2) dtype=int32>

In [6]:
x

<tf.Tensor 'Const_1:0' shape=(2, 1) dtype=int32>

In [7]:
y = tf.matmul(W, x)

In [10]:
with tf.Session() as sess:
    print(sess.run(y))
    print(sess.run(W))

[[3]
 [6]]
[[1 1]
 [2 2]]


## 神经网络的表达能力

- 无隐层神经网络
- 单隐层神经网络

- 不加激活函数的单隐层神经网络  
  $W'(Wx + b) + b' = (W'W)x + W'b + b'$
- [Universal approximation theorem](http://www.wikiwand.com/en/Universal_approximation_theorem)（单隐层）： $|f(x) - y| < \epsilon$
- Why go deep?  层数变多，参数个数变少，效果更好
  单层，每个神经元相对独立的
  多层，类似函数调用，第一层是非常小（底层）的函数；组件，后面的利用前面的组件；符合人类认知，从已有知识+关系学到更多知识；预测

### 深度的好处：结果复用

- Bengio 的电脑程序类比
- [icml09-ConvolutionalDeepBeliefNetworks.pdf](http://web.eecs.umich.edu/~honglak/icml09-ConvolutionalDeepBeliefNetworks.pdf)
- 深度的优势还没有理论上的完全定论。但是实际当中效果确实很好

### 过拟合

- [Overfitting slides11.pdf](http://work.caltech.edu/slides/slides11.pdf)
- 过拟合不一定是模型问题，也可能是数据量不够；数据很多时，参数很多也不容易过拟合；数据很少时，参数很少也可能过拟合



- 复杂模型拟合了数据当中的噪音
- 用什么模型？参数怎么定？
- 测试集（Test set）和验证集（Validation set）
  - 验证集：确定超参数

## 矩阵符号 Revisit

纠错：
- 视频3: 00:08 $w_{21}x_2$ 应为 $w_{21}x_1$
- 视频3: 06:15 行向量的两个矩阵等式 $w_{12}x_2$ 应为 $w_{12}x_1$

tf 假设输入是行向量，每个样本是一行

### 作业2

- 基础代码 tf_matrix.ipynb，将列向量表示形式，改成行向量表示形式
- 进一步，使用 `tf.layers.dense` 替换原有的矩阵乘法

## 统计语言模型存在的问题

- $p(w_3|w_1w_2)$
  - 数据中有很多的 $p(一|今天，星期), p(二|今天，星期)$
  - 数据中没有 $p(一|明天，星期), p(二|明天，星期)$
  - 类似于查表
  - 数据中是可以知道【今天】【明天】两个词很像
  - 怎么判断两个词很像？看他们有没有类似的【上下文】
  - 基于统计的 N-gram Language Model 没有办法【迁移】这种知识

## 神经网络语言模型

- 要素：模型的输出是一个概率分布，有 N 个词，就要有 N 个输出，还需要加和 = 1
- Sigmoid -> Softmax
- 要素：输入是上下文（比如前文）
- 最简单的做法：词表大小是 N，输入有 N 个神经元

In [15]:
c = tf.constant([1., 2, 3], dtype=tf.float32)

In [16]:
tf.nn.softmax(c)

<tf.Tensor 'Reshape_4:0' shape=(3,) dtype=float32>

In [17]:
with tf.Session() as sess:
    print(sess.run(tf.nn.softmax(c)))

[ 0.09003057  0.24472848  0.66524094]


In [18]:
import math

In [19]:
denom = math.exp(1) + math.exp(2) + math.exp(3)

In [21]:
math.exp(1) /denom

0.09003057317038046

纠错：视频4 13:00 和 13:45 是不是重复

In [22]:
import time
import tensorflow as tf

In [23]:
tf.reset_default_graph()

In [24]:
matrix = tf.random_normal([30000, 1])

In [29]:
# 直接告诉它哪些元素是 0 ，这里指定一个元素是 1，其他都是 0
# 存储空间很小，只存储非 0 元素对应的位置 和 值
# 和稠密矩阵或另一个稀疏矩阵做乘法时，有一个特殊的乘法，可以优化计算，把计算量优化到很小
x = tf.SparseTensor(indices=[[0, 0]], values=[1.0], dense_shape=[2000, 30000]) 
x_dense = tf.random_normal([2000, 30000]) # 稠密矩阵，大部分元素不是 0, Sparse tensor 大部分元素都是 0

### Sparse tensor * dense tensor

In [36]:
with tf.Session() as sess:
    t0 = time.time()
    # 假设 session 用 with，可以直接调用 eval 表示得到值，run graph 中必要的部分得到值
    tf.sparse_tensor_dense_matmul(x, matrix).eval()
    t1 = time.time()
    print(t1 - t0)

0.00494313240051


### Dense tensor * Dense tensor

In [39]:
with tf.Session() as sess:
    t0 = time.time()
    # 假设 session 用 with，可以直接调用 eval 表示得到值，run graph 中必要的部分得到值
    tf.matmul(x_dense, matrix).eval()
    t1 = time.time()
    print(t1 - t0)

1.14146113396


### Embedding lookup (矩阵取行)

### 作业3

为什么取这个矩阵的某一行，和这个词出现的时候做矩阵乘法，结果是一样的呢？

In [42]:
matrix1 = tf.random_normal([30000, 8])

In [43]:
with tf.Session() as sess:
    t0 = time.time()
    # 取矩阵的某一行或者某几行，matrix 就是 权重矩阵，第二个参数是 list，[0, 1] 表示 第 0 和 第 1 行
    print(tf.nn.embedding_lookup(matrix1, [0, 1]).eval())
    t1 = time.time()
    print("--------")
    print(t1 - t0)

[[ 0.50270391  0.51018757  0.24039407 -1.09312904  0.4385021  -0.75943923
  -1.26064861 -0.77937162]
 [-0.10846059  1.3035562   1.51551282  0.67917299 -0.70449454 -0.84641021
  -1.13484204  1.20230627]]
--------
0.0138111114502


## 为什么 NNLM 可以 迁移 知识

- 所有的样本进行全局优化
- 【今天】【明天】在很相似的上下文里面，他们的词向量也会比较像
- 【今天】有而【明天】没有的一些上下文，【明天】自然可以 Get 到